# Woolsey Fire data exploration

This notebook surveys satellite imagery related to the 2018 Woolsey Fire, including vegetation, climate, and topographic data.

In [ ]:
import os
from pathlib import Path
import warnings

import earthpy as et
import earthpy.plot as ep
import geopandas as gpd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rasterio.enums import Resampling
import rioxarray as rxr
import xarray as xr

from ea_drought_burn.config import CRS
from ea_drought_burn.utils import (
    copy_xr_metadata,
    load_nifc_fires,
    open_raster,
    plot_bands,
    plot_rgb,
    reproject_match
)




BOUND_KWARGS = {
    "facecolor": "none",
    "edgecolor": "black",
    "linewidth": 2
}




# Set default plotting parameters
plt.rc("font", size=18)
plt.rc("axes", labelsize=18)
plt.rc("xtick", labelsize=18)
plt.rc("ytick", labelsize=18)
plt.rc("legend", fontsize=18)
plt.rc("figure.constrained_layout", use=True, h_pad=12/72, w_pad=12/72)

# Set working directory to the earthpy data directory
os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data", "woolsey-fire"))

In [ ]:
def plot_four(bands, mask, *bounds, width=16, suptitle=None, **kwargs):
    """Plots bands in two columns"""
    
    # Scale fonts
    font_size = plt.rcParamsDefault["font.size"]
    plt.rc("font", size=font_size * width // 16)
    
    # Scale outline
    bound_kwargs = BOUND_KWARGS.copy()
    bound_kwargs["linewidth"] *= width // 16
    
    titles = [mask.format(year) for year in range(2013, 2017)]
    
    nrows = int(np.ceil(len(bands) / 2))
    height = (width * 0.7) / 2 * nrows

    fig, axes = plt.subplots(nrows, 2, figsize=(width, height))
    
    if suptitle:
        fig.suptitle(suptitle)
    
    axes_flat = []
    for row in axes:
        try:
            axes_flat.extend(row)
        except TypeError:
            axes_flat.append(row)

    for ax, title, band in zip(axes_flat, titles, bands):
        plot_bands(band, ax=ax, title=title, **kwargs)
        for bound in bounds:
            bound.plot(ax=ax, **bound_kwargs)
    
    # Reset font size to default
    plt.rcParams.update(plt.rcParamsDefault)

# Load boundaries and reprojection reference raster

The perimeter for the Woolsey Fire is from the National Interagency Fire Center.

In [ ]:
# Load the Woolsey Fire perimeter
woolsey_fire = gpd.read_file(os.path.join("shapefiles",
                                          "nifc_woolsey_perimeter",
                                          "2018-CAVNC-091023.shp")).to_crs(CRS)

# Set crop boundary
crop_bound = woolsey_fire.envelope

In [ ]:
# Load raster to use for reprojections
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr6.tif")
reproj_to = open_raster(path, crs=CRS, crop_bound=crop_bound)
reproj_to = reproj_to.where(reproj_to != -9999, np.nan)

# USGS topographic data

In [ ]:
# Read elevation data
path = os.path.join("usgs-terrain", "usgs_2016_18_merge_rs15m_fix.tif")
elevation = open_raster(path)
elevation = reproject_match(elevation, reproj_to)

# Read slope data
path = os.path.join("usgs-terrain", "usgs_2016_18_merge_rs15m_fix_slope.tif")
slope = open_raster(path)
slope = reproject_match(slope, reproj_to)

# Read aspect data
path = os.path.join("usgs-terrain", "usgs_2016_18_merge_rs15m_fix_aspect.tif")
aspect = open_raster(path)
aspect = reproject_match(aspect, reproj_to)

# Calculate folded aspect
folded_aspect = np.absolute(180 - np.absolute(aspect - 225))

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle("Woolsey Fire - Topography - USGS - 10 m")
plot_bands(elevation, ax=ax1, title="Elevation")
plot_bands(slope, ax=ax2, title="Slope")
plot_bands(aspect, ax=ax3, title="Aspect")
plot_bands(folded_aspect, ax=ax4, title="Folded aspect")

pass

**Caption:** Topographic data for the Woolsey Fire scar, including elevation, slope, aspect, and folded aspect. Folded aspect. calculated as *|180 - |aspect - 225||*, converts aspect into a linear parameter where hotness/dryness increases moving from 0° (for a northeast-facing slope) to 180° (for a southwest-facing slope) (McCune and Keon, 2002).

# Sentinel-2 pre- and post-fire images

Pre- and post-fire Sentinel-2 imagery is much better than the available Landsat images, which are obscured by smoke or clouds through early 2019.

In [ ]:
# Load pre-fire data from Sentinel-2 (ESA)
path = os.path.join("sentinel-2-imagery",
                    "3_aligned",
                    "L1C_T11SLT_A008633_20181031T184032.tif")
prefire = reproject_match(open_raster(path), reproj_to)

# Load post-fire data from Sentinel-2 (ESA)
path = os.path.join("sentinel-2-imagery",
                    "3_aligned",
                    "L1C_T11SLT_A018185_20181215T184316.tif")
postfire = reproject_match(open_raster(path), reproj_to)

# Plot the pre- and post-fire images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
fig.suptitle("Woolsey Fire - Sentinel-2 imagery - 30 m")
plot_rgb(prefire, ax=ax1, rgb=[3, 2, 1], stretch=True, title="2018-10-31")
plot_rgb(postfire, ax=ax2, rgb=[3, 2, 1], stretch=True, title="2018-12-15")

for ax in (ax1, ax2, ax3, ax4):
    woolsey_fire.plot(ax=ax, edgecolor="white", facecolor="none", linewidth=2)

I used the pre-fire Sentinel-2 data to calculate the following spectral indices:

+ NDVI
+ NDMI
+ NDWI
+ SAVI

The pre-fire indices generally show the same pattern as the fraction-alive plots.

In [ ]:
# Calculate pre-fire sprectral indices using Sentinel-2 data

ultrablue = prefire[0]
blue = prefire[1]        # L8 band 1
green = prefire[2]       # L8 band 2
red = prefire[3]         # L8 band 3
nir_705 = prefire[4]     # L8 band 4
nir_740 = prefire[5]
nir_783 = prefire[6]
nir_842 = prefire[7]     # L8 band 5
nir_865 = prefire[8]     # L8 band 5
swir_940 = prefire[9]
swir_1375 = prefire[10]  # L8 band 9
swir_1610 = prefire[11]  # L8 band 6
swir_2190 = prefire[12]  # L8 band 7

ndvi = (nir_842 - red) / (nir_842 + red)
ndwi = (nir_865 - swir_1610) / (nir_865 + swir_1610)
ndmi = (nir_842 - swir_1610) / (nir_842 + swir_1610)
savi = 1.5 * (nir_842 - red) / (nir_842 + red + 0.5)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))
fig.suptitle("Woolsey Fire - Sentinel-2 spectral indices - 30 m\n"
             "31 Oct 2018 (pre-fire)")

plot_bands(ndvi,
           ax=ax1,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="NDVI")

plot_bands(ndwi,
           ax=ax2,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="NDWI")

plot_bands(ndmi,
           ax=ax3,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="NDMI")

plot_bands(savi,
           ax=ax4,
           cmap="RdYlGn",
           vmin=-1,
           vmax=1,
           title="SAVI")

for ax in (ax1, ax2, ax3, ax4):
    woolsey_fire.plot(ax=ax, **BOUND_KWARGS)

In [ ]:
# Calculate NBR and dNBR
nir = prefire[8]
swir2 = prefire[12]
pre_nbr = (nir - swir2) / (nir + swir2)

nir = postfire[7]
swir2 = postfire[11]
post_nbr = (nir - swir2) / (nir + swir2)

dnbr = pre_nbr - post_nbr

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr,
           ax=ax,
           cmap="RdYlGn_r",
           title=("Woolsey Fire - Sentinel dNBR - 30 m\n"
                  "31 Oct 2018 - 15 Dec 2018"))
woolsey_fire.plot(ax=ax, **BOUND_KWARGS)

pass

In [ ]:
# Define dNBR classification bins
dnbr_class_bins = [-np.inf, -.1, .1, .27, .66, np.inf]
dnbr_class = xr.apply_ufunc(np.digitize, dnbr, dnbr_class_bins)

dnbr_class = dnbr_class.astype(np.float64)
dnbr_class = dnbr_class.rio.clip(woolsey_fire.geometry)

dnbr_cat_names = [
    "Increased Greenness",
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
]

nbr_colors = [
    "#7FFF00",
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
]
nbr_cmap = ListedColormap(nbr_colors)

# Plot the classfied dNBR data with a custom legend
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

plot_bands(dnbr_class,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("Woolsey Fire - Sentinel-2 classified dNBR - 30 m\n"
                  "31 Oct 2018 - 15 Dec 2018"),
           cbar=False,
           scale=False,
           ax=ax)
woolsey_fire.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names,
               bbox=(0.01, 0.99))

pass

# MTBS burn severity plots

MTBS provides normalized, field-validated dNBR plots that are more reliable than those that, like the one I calculated above, are made solely from satellite images.

In [ ]:
# Plot MTBS dNBR
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr.tif")
dnbr = open_raster(path, crs=CRS)
dnbr = dnbr.where(dnbr != -9999, np.nan)
dnbr /= 1000  # MTBS pixels are scaled by 1000
dnbr = reproject_match(dnbr, reproj_to)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr,
           ax=ax,
           cmap="RdYlGn_r",
           title=("Woolsey Fire - MTBS dNBR - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"))
woolsey_fire.plot(ax=ax, **BOUND_KWARGS)

pass

The MTBS dNBR plot shows fewer moderate- and high-severity pixels than the Sentinel-2 dNBR plot calculated above.

In [ ]:
# Plot MTBS classified dNBR
path = os.path.join("mtbs-burn-severity",
                    "ca3424011870020181108",
                    "ca3424011870020181108_20171215_20181215_dnbr6.tif")
dnbr_class = open_raster(path, crs=CRS, crop_bound=woolsey_fire.geometry)
dnbr_class = reproject_match(dnbr_class, reproj_to)
dnbr_class = dnbr_class.where(dnbr_class != -9999, np.nan)
dnbr_class = dnbr_class.where(dnbr_class != 6, np.nan)

dnbr_cat_names = [
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
    "Increased Greenness"
]

nbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]
nbr_cmap = ListedColormap(nbr_colors)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_bands(dnbr_class,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("Woolsey Fire - MTBS classified dNBR - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax)
woolsey_fire.plot(ax=ax, **BOUND_KWARGS)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names,
               bbox=(0.01, 0.99))

pass

# Woolsey Fire Stack

The original data included a second stack, WoolseyFireDataStack2, that included some Landsat data and BARC burn severity data, as well as the FAL data included in the SMM stack. All of that content is replicated elsewhere, and this file is not included in the final version of the data for this project.

+ **Landsat:** The WoolseyFireDataStack2 file contains two sets of Landsat files,
  one from 2017-10-24 and one from 2018-10-18. Both pre-date the Woolsey fire and
  include only bands 4 (red) and 7 (SWIR2). None of the spectral indices I'm aware
  of use those two bands.
+ **BARC:** BARC stands for Burned Area Reflectance Classification and is a 
  burn-severity index similar to dNBR. We decided to use the MTBS dNBR data
  instead.

# SMM Stack

The Santa Monica Mountain stack includes data about the vegetation and climate of the Santa Monica Mountain region in 2013-2016.

In [ ]:
# Open SMM stack
smm_stack = open_raster(
    os.path.join("aviris-climate-vegetation", "SMMDroughtstack.dat"),
    crs=CRS
)
smm_stack = smm_stack.where(smm_stack >= -1e38)
smm_stack = reproject_match(smm_stack, reproj_to)

In [ ]:
# Plot vegetation community
smm_community = smm_stack[0]

colors = [
    "black",
    "tab:green",
    "tab:olive",
    "tab:cyan",
    "tab:red",
    "tab:blue",
    "lightgray"
]

font_size = plt.rcParamsDefault["font.size"]
plt.rc("font", size=font_size * 2)

fig, ax = plt.subplots(figsize=(20, 20))
plot_bands(smm_community,
           ax=ax,
           title="SMM vegetation community map",
           cmap=ListedColormap(colors),
           cbar=False)

bound_kwargs = BOUND_KWARGS.copy()
bound_kwargs["linewidth"] *= 2
woolsey_fire.plot(ax=ax, **bound_kwargs)

# Draw legend
labels = [
    "No data",
    "Annual grass",
    "Chaparral",
    "Coastal sage scrub",
    "Oak woodland",
    "Riparian",
    "Substrate"
]
ep.draw_legend(ax.get_images()[0], 
               titles=labels,
               classes=range(len(labels)))

plt.rcParams.update(plt.rcParamsDefault)

## Fraction alive

The fraction-alive metric (FAL or RFAL) measures the ratio of healthy vegetation (green) to total vegetation (green plus brown).

In [ ]:
# Plot fraction alive
fal = smm_stack[1:5].copy()

plot_four(fal,
          "Fraction alive ({})",
          woolsey_fire,
          width=64,
          cmap="PiYG",
          vmin=0,
          vmax=1)

### dFAL

In [ ]:
# Plot year-on-year difference in fraction alive

dfal_13_14 = fal[0] - fal[1]
dfal_14_15 = fal[1] - fal[2]
dfal_15_16 = fal[2] - fal[3]
dfal_13_16 = fal[0] - fal[3]

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(64, 48))

font_size = plt.rcParamsDefault["font.size"]
plt.rc("font", size=font_size * 4)

plot_bands(dfal_13_14, ax=ax1, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2013-2014)")
plot_bands(dfal_14_15, ax=ax2, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2014-2015)")
plot_bands(dfal_15_16, ax=ax3, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2015-2016)")
plot_bands(dfal_13_16, ax=ax4, cmap="PiYG_r", vmin=-1, vmax=1, title="dFAL (2013-2016)")

bound_kwargs = BOUND_KWARGS.copy()
bound_kwargs["linewidth"] *= 4
for ax in (ax1, ax2, ax3, ax4):
    woolsey_fire.plot(ax=ax, **bound_kwargs)

plt.rcParams.update(plt.rcParamsDefault)

### Dieback threshold

Foster et al. (2017) use a threshold of 0.5431 to distinguish dead pixels from
live pixels based on a comparison of FAL and a field survey of oaks. The
correlation between the aerial and field surveys is poor and only looked at one
of the six vegetation communities in the study area, so that threshold is
poorly constrained. I've used a threshold of 0.5 below to identify dead pixels.

In [ ]:
# Plot dead pixels
half_dead = xr.where(fal < 0.5, True, False)
half_dead = half_dead.rio.clip(woolsey_fire.geometry)
half_dead = half_dead.where(half_dead != -2147483648, False)

plot_four(half_dead,
          ">50% dead ({})",
          woolsey_fire,
          width=64,
          cmap="Greys",
          cbar=False)

## Climate data

The SMM stack includes climate data from the PRISM project. In the plots below, green can generally be understood as being wetter/cooler than yellow.

In [ ]:
# Plot number of days with precipiation over 0.1 inches
days_precip = smm_stack[5:9]

plot_four(days_precip, 
          "Days precipitation ({})",
          woolsey_fire,
          cmap="summer_r",
          vmin=days_precip.min(),
          vmax=days_precip.max())

In [ ]:
# Plot maximum VPD
max_vpd = smm_stack[9:13]

plot_four(max_vpd, 
          "Max VPD ({})",
          woolsey_fire,
          cmap="summer_r",
          vmin=max_vpd.min(),
          vmax=max_vpd.max())

In [ ]:
# Plot minimum temperatue
min_temp = smm_stack[13:17]

plot_four(min_temp, 
          "Minimum temperature ({})",
          woolsey_fire,
          cmap="summer_r",
          vmin=min_temp.min(),
          vmax=min_temp.max())

In [ ]:
# Plot number of days over 95 F
heat_days_over_95 = smm_stack[17:21]

plot_four(heat_days_over_95, 
          "Heat days over 95 ({})",
          woolsey_fire,
          cmap="summer",
          vmin=heat_days_over_95.min(),
          vmax=heat_days_over_95.max())

In [ ]:
# Plot cumulative precipitation
cumulative_precip = smm_stack[21:25]
cumulative_precip = cumulative_precip.where(cumulative_precip >= 0, np.nan)

plot_four(cumulative_precip, 
          "Cumulative precipitation ({})",
          woolsey_fire,
          cmap="summer_r",
          vmin=cumulative_precip.min(),
          vmax=cumulative_precip.max())

In [ ]:
# Calculate area lost per vegetation type
# FIXME: Check this, numbers differ from previous work

print(f"Total area:   {2089 * 1681 * 15.6 **2 / 1e6:.0f} km2")
print("--")

comm = smm_stack[0]
for i in range(0, 7):
    subset = xr.where(comm == i, True, False).values
    color = colors[i]
    
    # Limit to vegetation (i.e., pixels with FAL calculated)
    comm_13_16 = dfal_13_16.where(subset)
    
    dfal = comm_13_16.mean().item()
    pixels = xr.where(comm_13_16, 1, 0).sum().item()
    pixel_km2 = comm_13_16.rio.resolution()[0] ** 2 * 1e-6
    
    area_km2 = dfal * pixels * pixel_km2
    
    print(labels[i])
    print(f"Pixels:          {pixels}")
    print(f"Area (km2):      {pixels * pixel_km2:.1f} km2")
    print(f"Area (acres):    {pixels * pixel_km2 / 0.004:.1f} acres")
    print(f"Dieback %:       {dfal:.3f}")
    print(f"Dieback (km2):   {area_km2:.1f} km2")
    print(f"Dieback (acres): {area_km2 / 0.004:.1f} acres")
    print("--")

# Previous fires

The area around the Woolsey Fire suffered ~24 wildfires between 2000 and 2018. This section summarizes the recent fire history and shows where that history may have impacted the severity of the Woolsey Fire.

In [ ]:
# List fires between 2000 and 2018 that intersect the Woolsey Fire perimeter
path = os.path.join("shapefiles",
                    "nifc_related_perimeters",
                    "RELATED_FIRE_PERIMTRS_2000_2018_DD83.shp")
nifc_fires = load_nifc_fires(path, crs=CRS)
nifc_fires = nifc_fires.sort_values(["perimeterd"]).reset_index(drop=True)

nifc_fires

In [ ]:
# Build array showing when areas of the scar last burned
last_burned = xr.where(smm_stack[0], -9999, -9999)
last_burned.rio.write_nodata(-9999, inplace=True)

woolsey_date = nifc_fires.iloc[-1].perimeterd
for i, row in nifc_fires.iterrows():
    days = (woolsey_date - row.perimeterd).days
    if days:
    
        # Create a mask based on last_burned with a nodata value of False
        mask = xr.where(last_burned, True, True)
        mask = mask.rio.write_nodata(0) 

        # Update the mask
        mask = mask.rio.clip([row.geometry], drop=False)

        last_burned = copy_xr_metadata(last_burned,
                                       xr.where(mask, days, last_burned))
    
last_burned = last_burned.where(last_burned != -9999, np.nan)

# Create plot comparing last burned to dNBR
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

plot_bands(dnbr_class,
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("Woolsey Fire - MTBS classified dNBR - 30 m\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax1)

plot_bands(last_burned,
           ax=ax2,
           title=("Woolsey Fire - Days since last fire - NIFC\n"
                  "2000-2018"))

for ax in (ax1, ax2):
    woolsey_fire.plot(ax=ax, **BOUND_KWARGS)

pass

**Caption:** MTBS dNBR plot (left) and areas of recent wildfires (right). Two areas dark green areas on the dNBR plot--one in the southeast corner and the other on the southwest edge of the fire scar--are difficult to distinguish based on vegetation, topography, or climate from nearby burned areas but do correspond to areas that have burned since 2000.


# Live fuel moisture content

Rao et al. (2020) https://www.sciencedirect.com/science/article/pii/S003442572030167X

In [ ]:
# Read LFMC data
lfmc = open_raster(
    os.path.join("_pending", "lfmc.tif"),
    crs=CRS
)
lfmc = reproject_match(lfmc, reproj_to)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plot_bands(lfmc, ax=ax)
woolsey_fire.plot(ax=ax, edgecolor="white", facecolor="none", linewidth=2)

pass

# Histograms

In [ ]:
data = {
    "FAL": fal[-1],
    "Slope": slope,
    "Folded Aspect": folded_aspect,
    "Elevation": elevation,
    "LFMC": lfmc,
    "NDMI": ndmi,
    "NDVI": ndvi,
    "NDWI": ndwi,
    "SAVI": savi,
    "Burned (2000-2018)": last_burned
}

# Limit data to pixels with good dNBR values
cond = (
    (dnbr_class.values > 1)
    & (ndvi > 0.1)
    & (ndwi > -0.2)
    & (lfmc > 40)
    & (lfmc < 100)
    & (slope < 55)
)
mask = np.where(cond, True, False)

subset = np.where(dnbr_class.values == 4, True, False)

kwargs = {"log": True}

for title, x in data.items():

    fig, ax = plt.subplots(1, 1, figsize=(8, 3))
    for i, x in enumerate([
        np.sort(np.ravel(x.values)),
        np.sort(np.ravel(x.values[mask])),
        np.sort(np.ravel(x.values[subset])),
        np.sort(np.ravel(x.values[mask & subset]))
    ]):
        if not i:
             _, bins, _ = ax.hist(x, **kwargs)
        else:
            ax.hist(x, bins=bins, **kwargs)
        
        ax.set_title(title)
        ax.legend(["a", "b", "c", "d"])

    pass